<a href="https://colab.research.google.com/github/dhruvchopra2003/Paper2/blob/main/LLM_2_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>